In [41]:
import requests
from bs4 import BeautifulSoup
import json
import string
import unicodedata
from datetime import datetime
import csv
import urllib

In [42]:
#category_id_list = [398,401,402,403,404,413,419,522,421,432,433,434,435,436,437,438,439,440,441,442,443,444,449,523,
#               343,348,357,364,369,374,375,517,528,390,391,392,393,394,395,396,518,451,456,457,464,465,470,473,
#               474,477,514,519,527,480,481,482,483,484,516,521,486,487,488,489,490,491,492,493,494,524,496,497,
#               498,499,500,501,502,505,525]
#region_list = ['beijing','shanghai','shenzhen','guangzhou','hangzhou','chengdu','xian','wuhan','ningbo']
category_id_list = [398,401,402]
region_list = ['ningbo']
fieldnames = ['tutor_id','topic_id','tutor_name','tutor_title','response_day','num_meet','num_wanna_see',
              'acceptance_rate','num_product','tutor_intro','len_tutor_intro','earliest_review_day','day_opened']

In [43]:
def get_topic_title_link(region, cat_id):
    page = 1
    while True:
        res = requests.get("http://www.zaih.com/topics/?category_id=%s&city=%s&page=%d" % (cat_id, region, page))
        soup = BeautifulSoup(res.text)
        for link in soup.select("a.topic-tutor-link"):
            yield link['href']
        if soup.select("span.icon.icon-next") == []:
            break
        page += 1

In [57]:
def crawl_tutor_page(region, cat_id, tutor_link):
    tutor_id = str(tutor_link[8:16])
    topic_id = str(tutor_link[23:31])
    res = requests.get("http://www.zaih.com%s" % tutor_link)
    soup = BeautifulSoup(res.text)
    tutor_name = soup.select("h1.tutor-name")[0].text    
    tutor_title = soup.select("p.tutor-title")[0].text
    temp = soup.select("span.highlight")
    response_day = temp[0].text
    num_meet = temp[1].text
    num_wanna_see = temp[2].text
    if len(temp) == 4:
        acceptance_rate = temp[3].text
    else:
        acceptance_rate = ""
    num_product = len(soup.select("li.topic-item"))
    tutor_intro = ""
    for intro in soup.select("div.about-tutor-summary"):
        tutor_intro += intro.text 
    tutor_intro = tutor_intro.strip()
    len_tutor_intro = len(tutor_intro)
    
    # convert unicode to string so it can be written to csv
    tutor_name = unicodedata.normalize('NFKD', tutor_name).encode('gb2312','ignore')
    tutor_intro = unicodedata.normalize('NFKD', tutor_intro).encode('ascii','ignore')
    tutor_title = unicodedata.normalize('NFKD', tutor_title).encode('ascii','ignore')
    response_day = unicodedata.normalize('NFKD', response_day).encode('ascii','ignore')
    num_meet = unicodedata.normalize('NFKD', num_meet).encode('ascii','ignore')
    num_wanna_see = unicodedata.normalize('NFKD', num_wanna_see).encode('ascii','ignore')
    if acceptance_rate != '':
        acceptance_rate = unicodedata.normalize('NFKD', acceptance_rate).encode('ascii','ignore')
    
    # get date of earliest review and the number of day_opened
    res2 = requests.get("http://www.zaih.com/apis/open/topic_reviews?tutor_id=85071906&offset=0&limit=10000&sort_by=latest")
    soup2 = BeautifulSoup(res2.text)
    temp_str_list = unicodedata.normalize('NFKD', soup2.select("body")[0].text).encode('ascii','ignore').split("date_created")
    if len(temp_str_list) == 1:
        earliest_review_day = "No review yet"
        day_opened = -1
    else:
        earliest_review_day = temp_str_list[-1][4:14]
        datetime_object = datetime.strptime(earliest_review_day,'%Y-%m-%d')
        time_now = datetime.now()
        day_opened = (time_now - datetime_object).days
        
    # crawl tutor image
    image_link = soup.select("img")[0]['src']
    urllib.urlretrieve("http:%s" % image_link, "tutor_image/%s_image.jpg" % tutor_id)
    
    
    print type(tutor_id)
    print type(topic_id)
    print type(tutor_name)
    print type(tutor_title)
    print type(response_day)
    print type(num_meet)
    print type(num_wanna_see)
    print type(acceptance_rate)
    print type(num_product)
    print type(tutor_intro)
    print type(len_tutor_intro)
    print type(earliest_review_day)
    print type(day_opened)
    # write into a csv file  
    cat_shop_list = {'tutor_id': tutor_id, 'topic_id': topic_id, 'tutor_name': tutor_name, 'region': region, 
                     'tutor_title': tutor_title, 'cat_id': cat_id,'response_day': response_day,'num_meet': num_meet, 
                     'num_wanna_see': num_wanna_see,'acceptance_rate': acceptance_rate, 'num_product': num_product, 
                     'tutor_intro': tutor_intro,'len_tutor_intro': len_tutor_intro, 'earliest_review_day': earliest_review_day, 'day_opened':day_opened}
    with open('file.csv', 'r+b') as f:
        header = next(csv.reader(f))
        dict_writer = csv.DictWriter(f, header, -1)
        dict_writer.writerow(cat_shop_list)

In [ ]:
# Not done
def crawl_product(tutor_link):
    tutor_id = tutor_link[8:16]
    res = requests.get("http://www.zaih.com%s" % tutor_link)
    soup = BeautifulSoup(res.text)
    for product in soup.select("li.topic-item"):
        # crawl sth
        # export to csv

In [ ]:
# not done
def crawl_review(tutor_link):

In [58]:
# main program starts

# for each pair of region and catalog_id
global_topic_title_link = []
for region in region_list:
    for cat_id in category_id_list:
        # reset the list
        topic_title_link = []
        
        # get all mentor page link under the same region and categories
        topic_title_link += get_topic_title_link(region, cat_id)
        global_topic_title_link += topic_title_link
        
        # crawl each mentor's page
        for tutor_link in topic_title_link:
            crawl_tutor_page(region, cat_id, tutor_link)
            
    # get unique tutor link, avoid crawling repeated product information
    # if memory overflow, write this to a file and then clear this
    global_topic_title_link = set(global_topic_title_link)

#for tutor_link in global_topic_title_link:
 #   crawl_product(tutor_link)
  #  crawl_review(tutor_link)

<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'str'>
<type 'int'>
<type 'str'>
<type 'int'>
<type 'str'>